In [ ]:
import argparse
import configparser
import dateutil
import json
import logging
import os
import sys

import pandas as pd
from pymongo import MongoClient

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "-c", "--config", type=str, default="settings.conf", help="Path to config file."
# )
# args = parser.parse_args()

# config_file = args.config

config_file = 'settings.conf'

config = configparser.RawConfigParser()
config.read(config_file)

df_header = [
    ('timestamp', 'timestamp'),
    ('user_name', 'author.name'),
    ('user_id', 'author.id'),
    ('bot', 'author.isBot'),
    ('channel', 'channel.name'),
    ('message', 'content'),
    ('message_id', 'id')
]

In [ ]:
def create_dateframe(query={}):
    mongo_client = MongoClient(config["mongodb"]["connection_string"])
    mongo_db = mongo_client[config["mongodb"]["db"]]
    mongo_coll = mongo_db[config["mongodb"]["collection"]]

    docs = mongo_coll.find(query)

    data = {
        'timestamp': [],
        'user_name': [],
        'user_id': [],
        'bot': [],
        'channel': [],
        'message': [],
        'message_id': [],
        'sentiment': []
    }

    for msg in docs:
        data['timestamp'].append(msg['timestamp'])
        data['user_name'].append(msg['author']['name'])
        data['user_id'].append(msg['author']['id'])
        data['bot'].append(msg['author']['isBot'])
        data['channel'].append(msg['channel']['name'])
        data['message'].append(msg['content'])
        data['message_id'].append(msg['id']),
        data['sentiment'].append(None)

    df = pd.DataFrame(data)
    df = df[:3]

    return df


dataframe = create_dateframe(query={})

print(dataframe)

In [ ]:
from google.cloud import language_v1beta2 as lang

client = lang.LanguageServiceClient()

document = {
    'content': '',
    'type_': lang.types.Document.Type.PLAIN_TEXT,
}

encoding_type = lang.EncodingType.UTF32

sentiment_column = []
for idx, msg in dataframe.iterrows():
    document['content'] = msg['message']
    print(document)

    response = client.analyze_entity_sentiment(
        request={
            'document': document,
            'encoding_type': encoding_type
        }
    )
    print(response)
    sentiment_column.append(response)
dataframe['sentiment'] = sentiment_column

print(dataframe)

In [ ]:
users = []
for name in dataframe['user_name'].unique():
    if '#' not in name:
        users.append(name)

print(users)

In [ ]:
for user in users:
    user_data = dataframe[dataframe['user_name'] == user]
    print(f"{user} - {user_data['message'].count()}")